Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

# Make a node

In [2]:
μ = rand(5, 5)
σ = rand(5, 5)
aa = RandomNode(:a, [:i, :j], Normal, μ, σ)
bb = RandomNode(:b, [:j, :k], Gamma, rand(5, 3), rand(5, 3))

VB.RandomNode{Distributions.Gamma}(:b,[:j,:k],5x3 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=0.6263186462630275, θ=0.9534418243171701)    …  Distributions.Gamma(α=0.3275565592854046, θ=0.5863923962387416)  
 Distributions.Gamma(α=0.03961130706474658, θ=0.40073635560976495)     Distributions.Gamma(α=0.30460199383506104, θ=0.10186046624088863)
 Distributions.Gamma(α=0.7707484790399473, θ=0.4643344472295512)       Distributions.Gamma(α=0.4498593894995624, θ=0.9799046203134645)  
 Distributions.Gamma(α=0.5939627954333706, θ=0.11547983427254782)      Distributions.Gamma(α=0.9802957128331022, θ=0.980047330210698)   
 Distributions.Gamma(α=0.09311145179897995, θ=0.6053485072397164)      Distributions.Gamma(α=0.8954091046484047, θ=0.8028325976531183)  )

In [3]:
cc = ConstantNode(:c, [:i, :j], rand(3, 4))

VB.ConstantNode{Float64}(:c,[:i,:j],3x4 Array{Float64,2}:
 0.902903  0.388126    0.59634   0.366495
 0.724881  0.00997577  0.492124  0.816894
 0.210572  0.342047    0.59424   0.950445)

In [4]:
cc = ConstantNode(rand(3, 4), [:i, :j])

VB.ConstantNode{Float64}(symbol("##const#7733"),[:i,:j],3x4 Array{Float64,2}:
 0.880535  0.534407  0.0241815  0.725902
 0.261593  0.853768  0.745726   0.326537
 0.870482  0.848399  0.153521   0.101801)

In [5]:
nodes = Node[aa, bb]
fi = get_structure(nodes...)
fi

VB.FactorInds([:i,:j,:k],[5,3,5],Dict(:a=>[1,2],:b=>[2,3]))

In [6]:
a[i, j] ~ Normal(μ, σ)

VB.RandomNode{Distributions.Normal}(:a,[:i,:j],5x5 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=0.6764640891912204, σ=0.7545066092957078)   …  Distributions.Normal(μ=0.17912217669270336, σ=0.20081243549418892)
 Distributions.Normal(μ=0.9967496495318295, σ=0.11531979151713645)     Distributions.Normal(μ=0.6534010257873324, σ=0.893354926929401)   
 Distributions.Normal(μ=0.6254554630345308, σ=0.08115473827572872)     Distributions.Normal(μ=0.30434172223395173, σ=0.9585711690985244) 
 Distributions.Normal(μ=0.24219095370981347, σ=0.2698105619792166)     Distributions.Normal(μ=0.19777310233027845, σ=0.23589127602496807)
 Distributions.Normal(μ=0.1183986151418468, σ=0.37314848935794465)     Distributions.Normal(μ=0.5701596514154066, σ=0.5719746362181353)  )

In [7]:
c[i, j] ~ Const(rand(3, 4))

VB.ConstantNode{Float64}(:c,[:i,:j],3x4 Array{Float64,2}:
 0.0508784  0.812961  0.110793   0.39773 
 0.832096   0.133756  0.398302   0.108764
 0.184122   0.234593  0.0608306  0.235719)

# Make a factor

In [8]:
dims = (10, 2)

μ[j] ~ Normal(rand(dims[2]), ones(dims[2]))
τ ~ Gamma(1, 1)
x[i, j] ~ Normal(rand(dims), ones(dims))

f = @factor LogNormalFactor x μ τ;

# Make a (generated) value function

In [9]:
value(f)

-34.832382724980974

In [10]:
g = @factor EntropyFactor x
value(g)

28.378770664093462

# Let's make a simple model

We want a simple model of inference for a normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\mu, \tau) \\
    \mu \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

## Make the nodes

We need to make nodes for all the random variables that will need to be updated (we can also create nodes for parameter arrays, but these will be converted automatically if we don't).

In [11]:
dims = (20, 6)

# note: it won't matter much how we initialize here
μ[j] ~ Normal(zeros(dims[2]), ones(dims[2]))
τ[j] ~ Gamma(1.1 * ones(dims[2]), ones(dims[2]))
μ0[j] ~ Const(zeros(dims[2]))
τ0[j] ~ Const(2 * ones(dims[2]))
a0[j] ~ Const(1.1 * ones(dims[2]))
b0[j] ~ Const(ones(dims[2]))

y[i, j] ~ Const(rand(dims));

Now make factors: We need a Normal factor for the observation model plus a prior and an entropy for each node.

In [12]:
obs = @factor LogNormalFactor y μ τ
μ_prior = @factor LogNormalFactor μ μ0 τ0
τ_prior = @factor LogGammaFactor τ a0 b0

VB.LogGammaFactor{1}(VB.RandomNode{Distributions.Gamma}(:τ,[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:a0,[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.FactorInds([:j],[6],Dict(:τ=>[1],:b0=>[1],:a0=>[1])),Dict(:τ=>:x,:b0=>:β,:a0=>:α))

In [13]:
value(obs), value(μ_prior), value(τ_prior)

(-174.17772843646506,-13.593072740907873,0.0449816833123928)

In [14]:
m = VBModel([μ, τ, μ0, τ0, a0, b0, y], [obs, μ_prior, τ_prior])

VB.VBModel(VB.Node[VB.RandomNode{Distributions.Normal}(:μ,[:j],[Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0)]),VB.RandomNode{Distributions.Gamma}(:τ,[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:μ0,[:j],[0.0,0.0,0.0,0.0,0.0,0.0]),VB.ConstantNode{Float64}(:τ0,[:j],[2.0,2.0,2.0,2.0,2.0,2.0]),VB.ConstantNode{Float64}(:a0,[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.ConstantNode{Float64}(:y,[:i,:j],20x6 Array{Float64,2}:
 0.384504    0.114404   0.951399    0.114746    0.0932579  0.815197 
 0.0866127   0.71266    0.268901    0.625316    0.152168   0.0191707
 0.533531    0.205955   0.0048390

In [15]:
[n.name for n in m.nodes]

7-element Array{Any,1}:
 :μ 
 :τ 
 :μ0
 :τ0
 :a0
 :b0
 :y 

In [16]:
[typeof(f) for f in m.factors]

3-element Array{DataType,1}:
 VB.LogNormalFactor{2}
 VB.LogNormalFactor{1}
 VB.LogGammaFactor{1} 

In [17]:
m.graph

Dict{VB.Node,Array{Tuple{VB.Factor{N},Symbol},1}} with 7 entries:
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…

In [18]:
naturals(obs, μ)

6-element Array{Tuple{Float64,Float64},1}:
 (10.747098263763775,-11.000000000000002)
 (11.827766830175905,-11.000000000000002)
 (12.989794210899973,-11.000000000000002)
 (9.101258377976134,-11.000000000000002) 
 (9.672210347578188,-11.000000000000002) 
 (11.794481875532338,-11.000000000000002)

In [19]:
naturals(obs, τ)

6-element Array{Tuple{Float64,Float64},1}:
 (10.0,13.302950010206423)
 (10.0,13.976622767663539)
 (10.0,14.450072618332413)
 (10.0,12.363247212945705)
 (10.0,12.98054512709643) 
 (10.0,14.136017606090698)

In [20]:
check_conjugate(τ, m)

true

In [21]:
check_conjugate(μ, m)

true

In [22]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)

In [23]:
update!(μ, m, Val{:conjugate})

In [24]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=-0.17882853266648782, σ=0.7071067811865476)
 Distributions.Normal(μ=0.5853195388586747, σ=0.7071067811865476)  
 Distributions.Normal(μ=1.406996979693105, σ=0.7071067811865476)   
 Distributions.Normal(μ=-1.3426130766542217, σ=0.7071067811865476) 
 Distributions.Normal(μ=-0.9388890672167934, σ=0.7071067811865476) 
 Distributions.Normal(μ=0.5617835217187213, σ=0.7071067811865476)  